### Importa módulos necesarios y establece configuraciones

In [ ]:
import pandas as pd
from time import time
import matplotlib.pyplot as plt
import pandas as pd
import funs
from IPython.display import display
import numpy as np
import seaborn as sns
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_rows = 10

### Lee los nodos en un diccionario de dataframes

In [ ]:
tablas={}
for nodo in [2,3,4,5,6,8]:
    tablas[nodo] = pd.read_csv('medidas_nodo%s.csv'%nodo,parse_dates=True,index_col=0)

### Grafica los valores de las variables de cada nodo en gráficas interactivas

for nodo in [2,3,4,5,6,8]:
    data = ([{'x': tablas[nodo].index,'y': tablas[nodo][col],'name': col}
             for col in tablas[nodo].columns[0:4]])
    layout = dict(title = 'NODO %s' %nodo,
                  yaxis = dict(title = '°C, %RH, %Pw, Nivel UV'),
                  xaxis = dict(title = 'FECHA')
                 )
    fig = dict(data=data, layout=layout)
    iplot(fig,layout)

### Aplica funs.separa: obtine para cada nodo una lista de DF con los registros obtenidos entre periodos de medición con separación de más de t minutos

In [ ]:
# Aplica la función separa_nodos, que calcula el tamaño de los dataframes entre periodos.
# Los periodos de medición corresponden a una separación entre registros  mayor a t minutos.
# t = 60*24, los registros se realizan cada minuto, de modo que t equivale a un día.
t = 60 * 24
nodo, periodos = {}, {}
for i in [2,3,4,5,6,8]:
    nodo[i], periodos[i] = funs.separa_datos(tablas[i],t)

### Presenta información acerca de los periodos de medición

In [ ]:
for i in periodos:    
    display('Nodo %s'%i,periodos[i])

### Aplica un filtro a los registros, cualquier periodo de medición con menos de mr registros es eliminado

In [ ]:
# mr es la cantidad mínima de registros
mr = 1e3
for i in periodos:
    borrar =[]
    for j in periodos[i].index:
        if periodos[i].iloc[j,2] < mr:
            borrar.append(j)
            nodo[i][j]=np.nan
    periodos[i] = periodos[i].drop(index=borrar)
    nodo[i] = [nodo[i][x] for x in periodos[i].index if not(x in borrar)]

### Presenta la información relacionada a los periodos de medición para cada nodo luego de aplicado el filtro de cantidad de registros

In [ ]:
for i in periodos:
    display('Nodo %s'%i,periodos[i])
    
#    Periodo 1: 2018-05-09 10:25:00 - 2018-05-10 14:36:00
#    Periodo 2: 2018-05-23 10:08:00 - 2018-05-26 07:00:00
#    Periodo 3: 2018-06-05 10:21:00 - 2018-06-08 18:40:00
#    Periodo 4: 2018-06-11 08:56:00 - 2018-06-14 03:45:00

#### Se evidencian tres periodos de medición para los cuales hay registros de los nodos 2, 3, 5, 6 y 8

### Se agregan los indíces de los registros faltantes y se asignan valores NaN a las variables

In [ ]:
for i in nodo:
    for j in range(len(nodo[i])):
        nodo[i][j] = nodo[i][j].asfreq('60S')

#### A continuación se presentan las gráficas de las variables durante tres periodos de medición. 
### Gráficas para el primer periodo

In [ ]:
T = 0
for i in [2,3,5,6]:
    ax=nodo[i][T].iloc[:,0:5].plot(figsize=[20,4],
                                  secondary_y='INTENSIDAD LUMÍNICA'
                                  ,mark_right=None)
    ax2=nodo[i][T].iloc[:,4].plot(secondary_y=True,color='mediumpurple')
    ax.set_ylabel('°C, %RH, %, Nivel UV')
    ax2.set_ylabel('Lux')
    ax.set_xlabel('FECHA')
    ax.set_title('Variables del Nodo %d en el Periodo de medición # %d'%(i,T+1))
    plt.tight_layout()
    plt.show()

#### Para el primer periodo los registros de los nodos 2, 3 y 5 presentan gran similitud, tanto para los valores medidos de cada variable como para el rango de fechas de los registros, se eceptúa el caso del sensor de PT550 del nodo 2, pues los valores de la variable intensidad luminica registrados en la noche están errados, sea por falla en el sensor o por alguna fuente de luz presente en su cercanía.
#### Fallas en los valores registrados por el nodo 6
- Se observan periodos de hasta tres horas sin registrar datos de las variables Humedad Relativa y Temperatura, correspondientes al sensor DHT11.
- Los valores de la variable Nivel UV permanencen constantes durante todo el periodo de medición, registrando una media de 7.52, lo que indica que este sensor se encontraba defectuoso.
- La media de los valores de Humedad de la tierra registrados es de 5.14, lo que indica que este sensor podría estar defectuoso o desconectado.
### Gráficas para el segundo periodo

In [ ]:
T = 1
for i in [2,3,5]:
    ax=nodo[i][T].iloc[:,0:5].plot(figsize=[20,4],
                                  secondary_y='INTENSIDAD LUMÍNICA'
                                  ,mark_right=None)
    ax2=nodo[i][T].iloc[:,4].plot(secondary_y=True,color='mediumpurple')
    ax.set_ylabel('°C, %RH, %, Nivel UV')
    ax2.set_ylabel('Lux')
    ax.set_xlabel('FECHA')
    ax.set_title('Variables del Nodo %d en el Periodo de medición # %d'%(i,T+1))
    plt.tight_layout()
    
ax=nodo[8][0].iloc[:,0:5].plot(figsize=[20,4],
                               secondary_y='INTENSIDAD LUMÍNICA'
                               ,mark_right=None)
ax2=nodo[8][0].iloc[:,4].plot(secondary_y=True,color='mediumpurple')
ax.set_ylabel('°C, %RH, %, Nivel UV')
ax2.set_ylabel('Lux')
ax.set_xlabel('FECHA')
ax.set_title('Variables del Nodo 8 en el Periodo de medición # 2')
plt.tight_layout()
plt.show()

#### Durante el segundo periodo de medición se presentaron fallas simultaneas en el sensor DTH11 del nodo 2 y variaciones anormales  en los registros de intensidad lumínica de los nodos 2, 3 y 5 crrespondientes a la mañana del 25 de mayo.
- Las diferencias respecto a los valores medidos en los nodos 2, 3 y 5 aunmentan en comparación a lo observado en el periodo de medición # 1.
#### El nodo 8 presentó fallas en todos sus sensores, salvo el orrespondiente a Humedad de la Tierra.

### Gráficas para el tercer periodo

In [ ]:
T = 2
for i in [2,3,5]:
    ax=nodo[i][T].iloc[:,0:5].plot(figsize=[20,4],
                                  secondary_y='INTENSIDAD LUMÍNICA'
                                  ,mark_right=None)
    ax2=nodo[i][T].iloc[:,4].plot(secondary_y=True,color='mediumpurple')
    ax.set_ylabel('°C, %RH, %Pw, Nivel UV')
    ax2.set_ylabel('Lux')
    ax.set_xlabel('FECHA')
    ax.set_title('Variables del Nodo %d en el Periodo de medición # %d'%(i,T+1))
    plt.tight_layout()
    
ax=nodo[8][1].iloc[:,0:5].plot(figsize=[20,4],
                               secondary_y='INTENSIDAD LUMÍNICA'
                               ,mark_right=None)
ax2=nodo[8][1].iloc[:,4].plot(secondary_y=True,color='mediumpurple')
ax.set_ylabel('°C, %RH, %Pw, Nivel UV')
ax2.set_ylabel('Lux')
ax.set_xlabel('FECHA')
ax.set_title('Variables del Nodo 8 en el Periodo de medición # 2')
plt.tight_layout()
plt.show()

#### Durante el tercer periodo de medición se presentaron las mayores similitudes entre las variables registradas en los nodos 2, 3 y 5.
- Los bajos niveles de humedad del suelo en todos los nodos corresponden a que han transcurrido varios días desde que se regó el cultivo por última vez y al redución de las precipitaciones en la región.

#### El nodo 8 no funcióno correctamente, los únicos registros correctos corresponden a las primeras 12 horas de medición.

##### A partir de la información anterior se establece que los nodos 6 y 8 no aportan información útil para el análisis de los datos, pues se evidencian demasiadas fallas en los valores. Además, se puede establecer que hay tres periodos de medición para los cuales se tienen registros simultaneos de los nodos 2, 3 y 5. 
- La variable Nivel UV no se va a considerar para el estudio, dado que la cantidad de valores medidos que corresponden a la incidencia de radiación solar directamente sobre los sensores se presentan en un reducido espacio de tiempo durante los días de medición, esto corresponde a la ubicación de los sensores a la altura de los frutos de cacao.

#### A continuación se procede a igualar los registros para los tres nodos teniendo en cuenta los siguientes rangos:
- Periodo 1: 2018-05-09 10:25:00 - 2018-05-10 14:36:00
- Periodo 2: 2018-05-23 10:08:00 - 2018-05-26 07:00:00
- Periodo 3: 2018-06-05 10:21:00 - 2018-06-08 18:40:00

### Se definen las fechas en las que inician y terminan los periodos de medición

In [ ]:
inicio = {0: '2018-05-09 10:25:00',
          1: '2018-05-23 10:08:00',
          2: '2018-06-05 14:41:00'}
fin = {0: '2018-05-10 14:36:00',
       1: '2018-05-26 07:00:00',
       2: '2018-06-08 18:39:00'}

### Se extraen los rangos definidos para cada periodo de medición

In [ ]:
node = {2:[], 3:[], 5:[]}
info = {2:[], 3:[], 5:[]}
for i in [2,3,5]:
    node[i]={}
    info[i]=pd.DataFrame(columns=['Registro Inicial', 'Registro Final',
                                  'Registrado', 'No Registrado',
                                  'Registrado(%)', 'No Registrado(%)','Duración'])
    for T in range(3):
        node[i][T], infon = funs.info_periodo(nodo[i][T],inicio[T],fin[T])
        node[i][T] = node[i][T].drop(columns=['NIVEL UV'])
        info[i] = pd.concat([info[i],infon])
    info[i] = info[i].reset_index(drop=True)    

In [ ]:
for i in [2,3,5]:
    display('Nodo %d'%i,info[i])

In [ ]:
# Presenta los registros del segundo periodo de medición de los nodos 2, 3 y 5 
T = 0
mag = {0:'°C', 1:'%RH', 2:'%', 3:'Nivel UV', 4:'lx'}
for i in range(4):
    plt.figure(figsize=[20,5])
    plt.title('%s DE LOS NODODS 2, 3 y 5 DURANTE EL PERIODO DE MEDICIÓN #%d'%(nodo[2][0].columns[i],T+1))
    plt.xlabel('FECHA')
    plt.ylabel(mag[i])
    plt.plot(node[2][0].iloc[:,i],label='NODO 2')
    plt.plot(node[3][0].iloc[:,i],label='NODO 3')
    plt.plot(node[5][0].iloc[:,i],label='NODO 5')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Presenta los registros del segundo periodo de medición de los nodos 2, 3 y 5 
T = 1
mag = {0:'°C', 1:'%RH', 2:'%', 3:'Nivel UV', 4:'lx'}
for i in range(4):
    plt.figure(figsize=[20,5])
    plt.title('%s DE LOS NODODS 2, 3 y 5 DURANTE EL PERIODO DE MEDICIÓN #%d'%(nodo[2][0].columns[i],T+1))
    plt.xlabel('FECHA')
    plt.ylabel(mag[i])
    plt.plot(node[2][T].iloc[:,i],label='NODO 2')
    plt.plot(node[3][T].iloc[:,i],label='NODO 3')
    plt.plot(node[5][T].iloc[:,i],label='NODO 5')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Presenta los registros del tercer periodo de los nodos 2, 3 y 5 
T = 2
mag = {0:'°C', 1:'%RH', 2:'%', 3:'Nivel UV', 4:'lx'}
for i in range(4):
    plt.figure(figsize=[20,5])
    plt.title('%s DE LOS NODODS 2, 3 y 5 DURANTE EL PERIODO DE MEDICIÓN #%d'%(nodo[2][0].columns[i],T+1))
    plt.xlabel('FECHA')
    plt.ylabel(mag[i])
    plt.plot(node[2][T].iloc[:,i],label='NODO 2')
    plt.plot(node[3][T].iloc[:,i],label='NODO 3')
    plt.plot(node[5][T].iloc[:,i],label='NODO 5')
    plt.legend()

### Se reemplazan los valores NaN con los valores registrados en columnas superiores

In [ ]:
for i in node:
    for T in range(3):
        node[i][T] = node[i][T].ffill()

### Se almacenan los registros de los nodos 2, 3, y 5 para los tres periodos de medición

In [ ]:
for i in node:
    for T in range(3):
        node[i][T] = node[i][T].reindex(columns = sorted(node[i][T].columns))
        (node[i][T]).to_csv('nodo%d_periodo%d.csv'%(i, T+1))